In [1]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('spinco.env')

#project variables
datapath=os.environ['DATAPATH']
cognipath=datapath+"\\COGNITION"
dreamspath=datapath+"\\DREAMS"
masspath=datapath+"\\MASS"


ModuleNotFoundError: No module named 'plotly'

## load DREAMS

In [2]:
signals, annotations, signalsMetadata = loadDREAMSSpindles(dreamspath)

SubjectId: 0001--------------
resampling from 100 to 200
SubjectId: 0003--------------
resampling from 50 to 200
SubjectId: 0006--------------
duration discrepancy, removing last 0.7 seconds


In [3]:
annotations.head()

,index,startTime,duration,channel,subjectId,labelerId,type,samplerate,stopTime,startInd,stopInd
0,0,282.24,0.72,C3-A1,0001,0001,spindle,200,282.96,56448,56592
1,1,311.72,1.54,C3-A1,0001,0001,spindle,200,313.26,62344,62652
2,2,340.28,0.72,C3-A1,0001,0001,spindle,200,341.00,68056,68200
3,3,366.83,0.65,C3-A1,0001,0001,spindle,200,367.48,73366,73496
4,4,373.74,0.50,C3-A1,0001,0001,spindle,200,374.24,74748,74848


In [4]:
signalsMetadata.head()

,filename,channel,subjectId,duration,samplerate,database
0,excerpt1.txt,C3-A1,0001,1800,200,DREAMS
1,excerpt2.txt,CZ-A1,0002,1800,200,DREAMS
2,excerpt3.txt,C3-A1,0003,1800,200,DREAMS
3,excerpt4.txt,CZ-A1,0004,1800,200,DREAMS
4,excerpt5.txt,CZ-A1,0005,1800,200,DREAMS


## spindle duration histograms

In [5]:
spindles=annotations[annotations.type=='spindle'].copy()

In [6]:
len(spindles)

764

In [7]:
#spindles longer than 4 seconds
print(len(spindles[spindles.duration>4])/len(spindles)) #50 parts per million
spindles[spindles.duration>4]

0.0


,index,startTime,duration,channel,subjectId,labelerId,type,samplerate,stopTime,startInd,stopInd


In [8]:
aux=annotations[["type","labelerId","subjectId"]].groupby(["type","labelerId","subjectId"],as_index=False).size()
px.bar(aux,x="subjectId",y="size",color="labelerId",barmode="group",facet_row="type",title="DREAMS annotations overview")

In [9]:
px.histogram(spindles,x='duration',title="DREAMS spindle duration histogram")

In [10]:
px.histogram(spindles,x='duration',facet_row="labelerId",title="DREAMS spindle duration histogram by expert")

In [11]:
px.histogram(spindles,x='duration',color="subjectId",facet_row="labelerId",title="DREAMS spindle duration histogram by expert and signal")

In [12]:
aux=spindles[['subjectId','startTime']].groupby('subjectId',as_index=False).size()
aux

,subjectId,size
0,0001,167
1,0002,112
2,0003,49
3,0004,69
4,0005,142
5,0006,159
6,0007,18
7,0008,48


In [13]:
aux=pd.merge(signalsMetadata,aux,on="subjectId")
aux["density"]=aux["size"]/(aux["duration"]/3600)

In [14]:
aux["density"]

0    334.0
1    224.0
2     98.0
3    138.0
4    284.0
5    318.0
6     36.0
7     96.0
Name: density, dtype: float64

In [15]:
np.mean(aux["density"])

191.0

In [16]:
np.std(aux["density"])

106.55045753069294

In [17]:
px.histogram(aux['size'][aux['size']>500],nbins=6)

In [18]:
m=np.mean(aux['size'])
s=np.std(aux['size'])

In [19]:
(1542>(m-s))and(1542<(m+s))

False

In [20]:
m-1.25*s

28.905964043316914

In [21]:
aux['is1.5sigma']=aux.apply(
    lambda row: (row['size']>(m-0.75*s))and(row['size']<(m+0.75*s)),
    axis=1)
aux['is2sigma']=aux.apply(
    lambda row: (row['size']>(m-s))and(row['size']<(m+s)),
    axis=1)
aux['is5/2sigma']=aux.apply(
    lambda row: (row['size']>(m-1.25*s))and(row['size']<(m+1.25*s)),
    axis=1)
aux['is3sigma']=aux.apply(
    lambda row: (row['size']>(m-1.5*s))and(row['size']<(m+1.5*s)),
    axis=1)
aux

,filename,channel,subjectId,duration,samplerate,database,size,density,is1.5sigma,is2sigma,is5/2sigma,is3sigma
0,excerpt1.txt,C3-A1,0001,1800,200,DREAMS,167,334.0,False,False,False,True
1,excerpt2.txt,CZ-A1,0002,1800,200,DREAMS,112,224.0,True,True,True,True
2,excerpt3.txt,C3-A1,0003,1800,200,DREAMS,49,98.0,False,True,True,True
3,excerpt4.txt,CZ-A1,0004,1800,200,DREAMS,69,138.0,True,True,True,True
4,excerpt5.txt,CZ-A1,0005,1800,200,DREAMS,142,284.0,False,True,True,True
5,excerpt6.txt,CZ-A1,0006,1800,200,DREAMS,159,318.0,False,False,True,True
6,excerpt7.txt,CZ-A1,0007,1800,200,DREAMS,18,36.0,False,False,False,True
7,excerpt8.txt,CZ-A1,0008,1800,200,DREAMS,48,96.0,False,True,True,True


In [22]:
spindles[['labelerId','subjectId','duration']].groupby(['labelerId']).describe()

duration                                                  
             count      mean       std   min   25%   50%   75%   max
labelerId                                                           
0001         355.0  0.819859  0.268467  0.39  0.61  0.76  0.96  1.89
0002         409.0  0.997531  0.035272  0.49  1.00  1.00  1.00  1.00

## criterium to remove outliers

In [23]:
#keep around 99% of annotations
len(spindles)*0.01

7.640000000000001

In [24]:
len(spindles)*0.01/2

3.8200000000000003

In [25]:
minDuration=0.3
maxDuration=2.9

In [26]:
len(spindles[spindles.duration>maxDuration])

0

In [27]:
len(spindles[spindles.duration<minDuration])

0

In [28]:
(len(spindles[spindles.duration>maxDuration])+len(spindles[spindles.duration<minDuration]))/len(spindles)

0.0

In [29]:
1-(len(spindles[spindles.duration>maxDuration])+len(spindles[spindles.duration<minDuration]))/len(spindles)

1.0

In [30]:
spindles['overMax']=spindles.duration>maxDuration
spindles['underMin']=spindles.duration<minDuration
spindles['discard']=spindles.overMax|spindles.underMin
spindles['keep']=(spindles.duration>minDuration)&(spindles.duration<maxDuration)
spindles

,index,startTime,duration,channel,subjectId,labelerId,type,samplerate,stopTime,startInd,stopInd,overMax,underMin,discard,keep
0,0,282.240,0.72,C3-A1,0001,0001,spindle,200,282.960,56448,56592,False,False,False,True
1,1,311.720,1.54,C3-A1,0001,0001,spindle,200,313.260,62344,62652,False,False,False,True
2,2,340.280,0.72,C3-A1,0001,0001,spindle,200,341.000,68056,68200,False,False,False,True
3,3,366.830,0.65,C3-A1,0001,0001,spindle,200,367.480,73366,73496,False,False,False,True
4,4,373.740,0.50,C3-A1,0001,0001,spindle,200,374.240,74748,74848,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,759,1731.695,1.00,CZ-A1,0006,0002,spindle,200,1732.695,346339,346539,False,False,False,True
760,760,1769.555,1.00,CZ-A1,0006,0002,spindle,200,1770.555,353911,354111,False,False,False,True
761,761,1780.175,1.00,CZ-A1,0006,0002,spindle,200,1781.175,356035,356235,False,False,False,True
762,762,1793.640,1.00,CZ-A1,0006,0002,spindle,200,1794.640,358728,358928,False,False,False,True


In [31]:
spindles[spindles.discard].groupby(['subjectId','labelerId']).describe()

Empty DataFrame
Columns: [(index, count), (index, mean), (index, std), (index, min), (index, 25%), (index, 50%), (index, 75%), (index, max), (startTime, count), (startTime, mean), (startTime, std), (startTime, min), (startTime, 25%), (startTime, 50%), (startTime, 75%), (startTime, max), (duration, count), (duration, mean), (duration, std), (duration, min), (duration, 25%), (duration, 50%), (duration, 75%), (duration, max), (samplerate, count), (samplerate, mean), (samplerate, std), (samplerate, min), (samplerate, 25%), (samplerate, 50%), (samplerate, 75%), (samplerate, max), (stopTime, count), (stopTime, mean), (stopTime, std), (stopTime, min), (stopTime, 25%), (stopTime, 50%), (stopTime, 75%), (stopTime, max), (startInd, count), (startInd, mean), (startInd, std), (startInd, min), (startInd, 25%), (startInd, 50%), (startInd, 75%), (startInd, max), (stopInd, count), (stopInd, mean), (stopInd, std), (stopInd, min), (stopInd, 25%), (stopInd, 50%), (stopInd, 75%), (stopInd, max)]
Index: []

[0 rows x 56 columns]

In [32]:
spindles[spindles.keep][['subjectId','labelerId','duration']].groupby(['subjectId','labelerId']).describe()

duration                                                   \
                       count      mean       std   min     25%    50%     75%   
subjectId labelerId                                                             
0001      0001          52.0  0.815385  0.271101  0.50  0.6450  0.760  0.8550   
          0002         115.0  0.991217  0.066308  0.49  1.0000  1.000  1.0000   
0002      0001          60.0  0.764500  0.211199  0.49  0.6050  0.730  0.8500   
          0002          52.0  1.000000  0.000000  1.00  1.0000  1.000  1.0000   
0003      0001           5.0  0.920000  0.347922  0.61  0.7000  0.760  1.0700   
          0002          44.0  1.000000  0.000000  1.00  1.0000  1.000  1.0000   
0004      0001          44.0  1.078636  0.336746  0.39  0.8500  1.010  1.3125   
          0002          25.0  1.000000  0.000000  1.00  1.0000  1.000  1.0000   
0005      0001          56.0  0.720179  0.181824  0.50  0.5625  0.720  0.8350   
          0002          86.0  1.000000  0.000000  1.00  1.0000  1.000  1.0000   
0006      0001          72.0  0.752083  0.223915  0.50  0.5900  0.700  0.9300   
          0002          87.0  1.000000  0.000000  1.00  1.0000  1.000  1.0000   
0007      0001          18.0  0.816111  0.219307  0.50  0.6100  0.815  1.0200   
0008      0001          48.0  0.865625  0.269309  0.50  0.6500  0.840  1.0000   

                           
                      max  
subjectId labelerId        
0001      0001       1.67  
          0002       1.00  
0002      0001       1.46  
          0002       1.00  
0003      0001       1.46  
          0002       1.00  
0004      0001       1.80  
          0002       1.00  
0005      0001       1.28  
          0002       1.00  
0006      0001       1.30  
          0002       1.00  
0007      0001       1.22  
0008      0001       1.89